# Data Munging

Data wrangling, sometimes referred to as data munging, is the process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics.

In [0]:
import pandas as pd  #importing pandas for data processing 
import numpy as np   #importing numpy for mathematical computation


import re            #importing Regular Expressions

    
import spacy         #importing spacy for natural language processing

nlp = spacy.load('en_core_web_sm')  #loading english grammer and rules

from scipy import stats    #importing stats for Chi-Square test


import warnings
warnings.filterwarnings("ignore")

In [0]:
data = pd.read_excel('DFP.xlsx', sheet_name = 'Data')

#Task 1

Dataset: ​Google Ad Manager Data ( DFP.xlsx ) 
1. Create the following columns, from the ​Ad_unit_name column in the dataset:

   a. amp_or_non_amp b. story c. position 
 
Example: 1.1 CarToq_ad_first_story_pos_top (122380182)        story = first, position = top, Nonamp  1.2 amp-cartoq-bottom (21684306640)       story = None, position = bottom, Amp 

In [0]:
data.head(10)

,DATE,DAY,AD_UNIT_NAME,ORDER_NAME,ADVERTISER_NAME,LINE_ITEM_NAME,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM
0,2018-01-01,1,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,Adsense house ad,74755,72602,0,0.0000,3630.100000,50.000000
1,2018-01-01,1,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,Adsense backup ad,13683,13241,0,0.0000,397.230000,30.000000
2,2018-01-01,1,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,CarToq ad first story pos first,86739,84928,1375,0.0162,5910.674200,69.596296
3,2018-01-01,1,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,Adx Native Ad,94482,91685,1277,0.0139,6124.947976,66.804253
4,2018-01-01,1,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,Adsense house ad,69605,67661,0,0.0000,3383.050000,50.000000
5,2018-01-01,1,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,Adsense backup ad,68194,66383,0,0.0000,1991.490000,30.000000
6,2018-01-01,1,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,CarToq ad first story pos second,60595,59311,826,0.0139,2960.428679,49.913653
7,2018-01-01,1,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,Adx Native Ad,54612,53099,677,0.0127,2637.101197,49.663858
8,2018-01-01,1,CarToq_ad_first_story_pos_bottom (122380542),CarToq Adx,Google Adx,Adsense house ad,18682,18116,0,0.0000,905.800000,50.000000
9,2018-01-01,1,CarToq_ad_first_story_pos_bottom (122380542),CarToq Adx,Google Adx,Adsense backup ad,87592,84953,0,0.0000,2548.590000,30.000000


In [0]:
data.shape

(50282, 12)

In [0]:
data.tail()

,DATE,DAY,AD_UNIT_NAME,ORDER_NAME,ADVERTISER_NAME,LINE_ITEM_NAME,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM
50277,2018-07-31,2,amp-cartoq-afterrelated (21719659148),Teads-Outstream-Video_monthly_11-06-2018,Teads.tv,Teads AMP Bottom Ad Unit,59652,45084,0,0.0000,2705.040000,60.000000
50278,2018-07-31,2,amp-cartoq-afterrelated (21719659148),Test Order,Internal Ad,Cartoq House ad,1117,862,0,0.0000,0.000000,0.000000
50279,2018-07-31,2,amp-cartoq-afterrelated (21719659148),Google Adsense - Cartoq English,Google Adsense,Adsense house ad - 300x250-1 - Cartoq Non AMP,203,152,0,0.0000,1.520000,10.000000
50280,2018-07-31,2,amp-cartoq-afterrelated (21719659148),Google Adsense - Cartoq English,Google Adsense,Adsense house ad - 336x280-1 - Cartoq Non AMP,45183,34270,0,0.0000,685.400000,20.000000
50281,2018-07-31,2,amp-cartoq-afterrelated (21719659148),Droom-Cartoq-PG,Droom,Droom-Cartoq-PG-Deal,44481,33802,47,0.0014,1879.494536,55.603057


### Initialising the columns
  1. Postiton
  2. amp_or_non_amp
  3. story

In [0]:
data['position'] = ''
data['amp_or_non_amp'] = ''
data['story'] = ''

### Spliting the text values based on the required positions and storing the data into particular column values

In [0]:
for i in range(data.shape[0]):
    if re.findall('amp', data['AD_UNIT_NAME'].iloc[i]):
        data['position'].iloc[i] = data['AD_UNIT_NAME'].iloc[i].split('-')[-1].split(' ')[0]
        data['amp_or_non_amp'].iloc[i] = 'Amp'
        data['story'] = 'None'
    else:
        data['position'].iloc[i] = data['AD_UNIT_NAME'].iloc[i].split('_')[-1].split(' ')[0]
        data['amp_or_non_amp'] = 'Nonamp'
        data['story'] = data['AD_UNIT_NAME'].iloc[i].split('_story')[0].split('_')[-1]

## Initialising the function to map dates with days

In [0]:
def days(number):
    dic = {1:'Monday',
          2:'Tuesday',
          3:'Wednesday',
          4:'Thursday',
          5:'Friday',
          6:'Saturday',
          7:'Sunday'}
    return dic[number]

## Using lambda functions to apply above function on the day data column

In [0]:
data['DAY'] = data['DAY'].apply(lambda x:days(x))

In [0]:
data.head(10)

,DATE,DAY,AD_UNIT_NAME,ORDER_NAME,ADVERTISER_NAME,LINE_ITEM_NAME,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM,position,amp_or_non_amp,story
0,2018-01-01,Monday,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,Adsense house ad,74755,72602,0,0.0000,3630.100000,50.000000,top,Nonamp,None
1,2018-01-01,Monday,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,Adsense backup ad,13683,13241,0,0.0000,397.230000,30.000000,top,Nonamp,None
2,2018-01-01,Monday,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,CarToq ad first story pos first,86739,84928,1375,0.0162,5910.674200,69.596296,top,Nonamp,None
3,2018-01-01,Monday,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,Adx Native Ad,94482,91685,1277,0.0139,6124.947976,66.804253,top,Nonamp,None
4,2018-01-01,Monday,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,Adsense house ad,69605,67661,0,0.0000,3383.050000,50.000000,middle,Nonamp,None
5,2018-01-01,Monday,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,Adsense backup ad,68194,66383,0,0.0000,1991.490000,30.000000,middle,Nonamp,None
6,2018-01-01,Monday,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,CarToq ad first story pos second,60595,59311,826,0.0139,2960.428679,49.913653,middle,Nonamp,None
7,2018-01-01,Monday,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,Adx Native Ad,54612,53099,677,0.0127,2637.101197,49.663858,middle,Nonamp,None
8,2018-01-01,Monday,CarToq_ad_first_story_pos_bottom (122380542),CarToq Adx,Google Adx,Adsense house ad,18682,18116,0,0.0000,905.800000,50.000000,bottom,Nonamp,None
9,2018-01-01,Monday,CarToq_ad_first_story_pos_bottom (122380542),CarToq Adx,Google Adx,Adsense backup ad,87592,84953,0,0.0000,2548.590000,30.000000,bottom,Nonamp,None


## Loading the Actual_eCPM Data set

 Here, eCPM for some data is incorrect, Merge ​Actual_eCPM         data from the other sheet ( Actual_eCPM.csv ) such that new           column for ​Actual_eCPM is created. Create another column        Actual_Revenue​ which is defined as:  Actual_Revenue = Total Impression * Actual_eCPM 
 
(if not provided, take the given revenue in the dataset as           Actual_Revenue to fill the remaining ​Actual_Revenue column       data with it.)  
 
After completing the above 3 tasks save the results as          DFP_solution.csv 

In [0]:
actual_epcm = pd.read_csv('Actual_eCPM.csv')

In [0]:
actual_epcm.head()

,LINE_ITEM_NAME,Actual_eCPM
0,Adsense house ad,-
1,Adsense backup ad,-
2,CarToq ad first story pos first,-
3,Adx Native Ad,-
4,CarToq ad first story pos second,-


## Merging the Actual_ecpm Dataset with DFP data Set

In [0]:
data = pd.merge(actual_epcm, data, on = 'LINE_ITEM_NAME')
data.head()

,LINE_ITEM_NAME,Actual_eCPM,DATE,DAY,AD_UNIT_NAME,ORDER_NAME,ADVERTISER_NAME,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM,position,amp_or_non_amp,story
0,Adsense house ad,-,2018-01-01,Monday,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,74755,72602,0,0.0,3630.10,50.0,top,Nonamp,None
1,Adsense house ad,-,2018-01-01,Monday,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,69605,67661,0,0.0,3383.05,50.0,middle,Nonamp,None
2,Adsense house ad,-,2018-01-01,Monday,CarToq_ad_first_story_pos_bottom (122380542),CarToq Adx,Google Adx,18682,18116,0,0.0,905.80,50.0,bottom,Nonamp,None
3,Adsense house ad,-,2018-01-01,Monday,CarToq_ad_second_story_pos_top (122380662),CarToq Adx,Google Adx,55769,54252,0,0.0,2712.60,50.0,top,Nonamp,None
4,Adsense house ad,-,2018-01-01,Monday,CarToq_ad_second_story_pos_middle (122380782),CarToq Adx,Google Adx,10634,10378,0,0.0,518.90,50.0,middle,Nonamp,None


In [0]:
data['Actual_eCPM'] = data['Actual_eCPM'].replace('-', 0.0)
data['Actual_eCPM'] = data['Actual_eCPM'].apply(lambda x:float(x))

In [0]:
data['eCPM'] = data['eCPM'].replace('-', 0.0)

In [0]:
data['Actual_eCPM'] = data['Actual_eCPM'] + data['eCPM']

In [0]:
data.head()

,LINE_ITEM_NAME,Actual_eCPM,DATE,DAY,AD_UNIT_NAME,ORDER_NAME,ADVERTISER_NAME,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM,position,amp_or_non_amp,story
0,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,74755,72602,0,0.0,3630.10,50.0,top,Nonamp,None
1,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,69605,67661,0,0.0,3383.05,50.0,middle,Nonamp,None
2,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_first_story_pos_bottom (122380542),CarToq Adx,Google Adx,18682,18116,0,0.0,905.80,50.0,bottom,Nonamp,None
3,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_second_story_pos_top (122380662),CarToq Adx,Google Adx,55769,54252,0,0.0,2712.60,50.0,top,Nonamp,None
4,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_second_story_pos_middle (122380782),CarToq Adx,Google Adx,10634,10378,0,0.0,518.90,50.0,middle,Nonamp,None


In [0]:
data['Actual_Revenue'] = data['Actual_eCPM'] * data['Impressions']

## Exporting data into csv File

In [0]:
data.to_csv('DFP_solution.csv', index = False)

Identify the best-performing Ad position in terms of eCPM and          revenue, separately in amp and non-amp case. ( hint : while           merging the data keep in mind that [ eCPM = Actual Revenue *             1000 / Total Impressions ] and [ CTR = clicks / Total            impressions] , aggregating directly won’t help.)  Submit this result as ​Adpos.csv

In [0]:
amp = data[data['amp_or_non_amp'] == 'Amp']
non_amp = data[data['amp_or_non_amp'] == 'Nonamp']

In [0]:
amp.sort_values(by = ['Actual_Revenue', 'Actual_eCPM'], 
                ascending = False,
                inplace = True)

In [0]:
amp.head()

,LINE_ITEM_NAME,Actual_eCPM,DATE,DAY,AD_UNIT_NAME,ORDER_NAME,ADVERTISER_NAME,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM,position,amp_or_non_amp,story,Actual_Revenue
31829,Teads AMP Bottom Ad Unit,261.000000,2018-07-31,Tuesday,amp-cartoq-afterrelated (21719659148),Teads-Outstream-Video_monthly_11-06-2018,Teads.tv,59652,45084,0,0.0000,2705.040000,60.000000,afterrelated,Amp,None,1.176692e+07
33590,Droom-Cartoq-PG-Deal,55.603057,2018-07-31,Tuesday,amp-cartoq-afterrelated (21719659148),Droom-Cartoq-PG,Droom,44481,33802,47,0.0014,1879.494536,55.603057,afterrelated,Amp,None,1.879495e+06
28209,Adsense house ad - 336x280-1 - Cartoq Non AMP,29.360000,2018-07-31,Tuesday,amp-cartoq-afterrelated (21719659148),Google Adsense - Cartoq English,Google Adsense,45183,34270,0,0.0000,685.400000,20.000000,afterrelated,Amp,None,1.006167e+06
18031,CarToq Adx All Story Pages,46.335601,2018-07-31,Tuesday,amp-cartoq-afterrelated (21719659148),CarToq Adx,Google Adx,16124,12144,113,0.0093,562.699543,46.335601,afterrelated,Amp,None,5.626995e+05
14360,AMP pages adx,45.942596,2018-07-31,Tuesday,amp-cartoq-afterrelated (21719659148),CarToq Adx,Google Adx,16339,12153,94,0.0077,558.340375,45.942596,afterrelated,Amp,None,5.583404e+05


In [0]:
non_amp.head()

,LINE_ITEM_NAME,Actual_eCPM,DATE,DAY,AD_UNIT_NAME,ORDER_NAME,ADVERTISER_NAME,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM,position,amp_or_non_amp,story,Actual_Revenue
0,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_first_story_pos_top (122380182),CarToq Adx,Google Adx,74755,72602,0,0.0,3630.10,50.0,top,Nonamp,None,3630100.0
1,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_first_story_pos_middle (122380422),CarToq Adx,Google Adx,69605,67661,0,0.0,3383.05,50.0,middle,Nonamp,None,3383050.0
2,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_first_story_pos_bottom (122380542),CarToq Adx,Google Adx,18682,18116,0,0.0,905.80,50.0,bottom,Nonamp,None,905800.0
3,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_second_story_pos_top (122380662),CarToq Adx,Google Adx,55769,54252,0,0.0,2712.60,50.0,top,Nonamp,None,2712600.0
4,Adsense house ad,50.0,2018-01-01,Monday,CarToq_ad_second_story_pos_middle (122380782),CarToq Adx,Google Adx,10634,10378,0,0.0,518.90,50.0,middle,Nonamp,None,518900.0


In [0]:
pd.concat([amp, non_amp]).to_csv('Adpos.csv', index = False)

# Task 2

**Statistical Analysis: Dataset: ​Google Ad Manager Data ( DFP.xlsx )** 
In this task, we want to test your familiarity with different statistical and              modeling concepts.  1. Analyze the dataset in as many ways possible ways, including          multivariate analysis, to generate insights for predictive       modeling. (Other suggestions for analysis – [ Correlation,        Covariance, ANOVA, Regression analysis, Hypothesis testing:      Student’s t-test, chi-square test (Generate Null and Alternate        hypothesis and find the significant relation)]. 
 
 

In [0]:
data.describe()

,Actual_eCPM,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM,Actual_Revenue
count,50126.000000,50126.000000,50126.000000,50126.000000,50126.000000,50126.000000,50126.000000,5.012600e+04
mean,80.844832,11243.377628,10426.028548,49.730220,0.001969,455.065833,76.816432,4.790261e+05
std,199.693220,29378.322110,27910.566290,249.179818,0.012978,1569.102097,199.198890,1.625717e+06
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,23.170000,4.000000,3.000000,0.000000,0.000000,0.207574,20.000000,2.095400e+02
50%,44.623991,297.000000,240.000000,0.000000,0.000000,2.662000,44.071403,3.614080e+03
75%,87.640000,7486.750000,6551.750000,1.000000,0.001500,206.419435,82.940000,2.322675e+05
max,10000.000000,521616.000000,498987.000000,6208.000000,1.000000,43767.803350,10000.000000,4.376780e+07


In [0]:
data.isnull().sum()

LINE_ITEM_NAME     0
Actual_eCPM        0
DATE               0
DAY                0
AD_UNIT_NAME       0
ORDER_NAME         0
ADVERTISER_NAME    0
Tags_served        0
Impressions        0
Clicks             0
CTR                0
Revenue            0
eCPM               0
position           0
amp_or_non_amp     0
story              0
Actual_Revenue     0
dtype: int64

In [0]:
data.corr()

,Actual_eCPM,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM,Actual_Revenue
Actual_eCPM,1.000000,-0.067260,-0.065285,-0.023283,-0.012986,-0.034614,0.995882,-0.027992
Tags_served,-0.067260,1.000000,0.988959,0.742607,0.079483,0.890558,-0.064357,0.875511
Impressions,-0.065285,0.988959,1.000000,0.755131,0.080628,0.901271,-0.062203,0.884587
Clicks,-0.023283,0.742607,0.755131,1.000000,0.122872,0.880591,-0.019758,0.847631
CTR,-0.012986,0.079483,0.080628,0.122872,1.000000,0.095444,-0.013038,0.090952
Revenue,-0.034614,0.890558,0.901271,0.880591,0.095444,1.000000,-0.032851,0.979969
eCPM,0.995882,-0.064357,-0.062203,-0.019758,-0.013038,-0.032851,1.000000,-0.033514
Actual_Revenue,-0.027992,0.875511,0.884587,0.847631,0.090952,0.979969,-0.033514,1.000000


In [0]:

chi2_stat, p_val, dof, ex = stats.chi2_contingency(data[['Actual_eCPM','Tags_served','Impressions','Clicks','CTR','Revenue','eCPM','Actual_Revenue']])

In [0]:
print("===Chi2 Stat===")
print(chi2_stat)
print("\n")
print("===Degrees of Freedom===")
print(dof)
print("\n")
print("===P-Value===")
print(p_val)
print("\n")
print("===Contingency Table===")
print(ex)

===Chi2 Stat===
10477215160.238047


===Degrees of Freedom===
350875


===P-Value===
0.0


===Contingency Table===
[[6.09722925e+02 8.47963310e+04 7.86319732e+04 ... 3.43205701e+03
  5.79341170e+02 3.61276260e+06]
 [5.68218659e+02 7.90241855e+04 7.32794398e+04 ... 3.19843449e+03
  5.39905012e+02 3.36683932e+06]
 [1.52157736e+02 2.11611164e+04 1.96227869e+04 ... 8.56477597e+02
  1.44575900e+02 9.01573085e+05]
 ...
 [3.19760567e-01 4.44702368e+01 4.12374265e+01 ... 1.79989377e+00
  3.03827285e-01 1.89466226e+03]
 [3.57114075e-01 4.96651217e+01 4.60546638e+01 ... 2.01015218e+00
  3.39319513e-01 2.11599125e+03]
 [5.38190757e-01 7.48480986e+01 6.94069379e+01 ... 3.02941105e+00
  5.11373364e-01 3.18891641e+03]]


In [0]:
data.cov()

,Actual_eCPM,Tags_served,Impressions,Clicks,CTR,Revenue,eCPM,Actual_Revenue
Actual_eCPM,3.987738e+04,-3.945899e+05,-3.638717e+05,-1.158532e+03,-0.033654,-1.084589e+04,3.961485e+04,-9.087411e+06
Tags_served,-3.945899e+05,8.630858e+08,8.109122e+08,5.436245e+06,30.303875,4.105259e+07,-3.766232e+05,4.181512e+10
Impressions,-3.638717e+05,8.109122e+08,7.789997e+08,5.251747e+06,29.204702,3.947073e+07,-3.458314e+05,4.013782e+10
Clicks,-1.158532e+03,5.436245e+06,5.251747e+06,6.209058e+04,0.397342,3.443009e+05,-9.807218e+02,3.433717e+08
CTR,-3.365436e-02,3.030388e+01,2.920470e+01,3.973421e-01,0.000168,1.943567e+00,-3.370522e-02,1.918910e+03
Revenue,-1.084589e+04,4.105259e+07,3.947073e+07,3.443009e+05,1.943567,2.462081e+06,-1.026799e+04,2.499819e+09
eCPM,3.961485e+04,-3.766232e+05,-3.458314e+05,-9.807218e+02,-0.033705,-1.026799e+04,3.968020e+04,-1.085324e+07
Actual_Revenue,-9.087411e+06,4.181512e+10,4.013782e+10,3.433717e+08,1918.909684,2.499819e+09,-1.085324e+07,2.642954e+12


#Task 3

Text Processing and Sentiment Analysis: Dataset: ​Review data ( Review_Data_All.xlsx ) 
 1. Basic Text Processing (Case Conversions, Punctuation      Removal, Stop word Removal, Stemming, Lemmatization)
 2. Top 10 High-frequency words 
 3. Use a polarity score for the cleaned review data to calculate the            Percentage of Positive and Negative Reviews. (use polarity        score threshold of 0 to classify positive and negative)
 4. Identify Most Positive, Most Negative and a Neutral review         based on the score 
 5. Use Polarity score of 0 to classify reviews Positive and          Negative. Use this as the target variable. [Binary Encode] 
 6. Use logistic regression to identify the words with high         coefficients value. 
 

In [0]:
reviews = pd.read_excel('Review_Data_All.xlsx', sheet_name = 'Review Data')

In [0]:
reviews.head()

,Rating,Review Date,Review Paragarph,Review Title,Site
0,1,2019-05-01,"I went to 24/7 in the Lalit Hotel, with its fi...","horrendous Service, never recommended.",TripAdvisor
1,5,2019-05-01,"Must visit, great staff , great managment , fo...",Perfect dine,TripAdvisor
2,4,2019-03-01,Food is very good. Breakfast very good variety...,GOOD FOOD,TripAdvisor
3,5,2019-04-01,Amazing staff My purpose of visit was busines...,Work from Home,TripAdvisor
4,1,2019-04-01,Hi You must be wondering why I have tested th...,Worst five star restaurant - tested many times,TripAdvisor


In [0]:
def natural_language(text):
    text = text.lower()
    text = re.sub('[0-9]*','',text)
    docx = nlp(text)
    temp = []
    for i in docx:
        if i.is_stop!=True and i.is_punct!=True:
            temp.append(i.lemma_)
    return ' '.join(temp)

In [0]:
reviews['cleaned_text'] = reviews['Review Paragarph'].apply(lambda x:natural_language(x))

In [0]:
words = ' '.join(reviews['cleaned_text'].values.tolist())
words = words.split(' ')
words = [x for x in words if x]

dic = {}
for i in words:
    if i not in dic.keys():
        dic[i] = 1
    else:
        dic[i] = dic[i] + 1

In [0]:
temp = pd.DataFrame()
temp['words'] = dic.keys()
temp['count'] = dic.values()
temp.sort_values(by = 'count', ascending = False, inplace = True)
temp = temp.head(10)
temp

,words,count
19,food,164
21,good,152
5,service,125
27,buffet,96
13,staff,87
12,restaurant,85
29,breakfast,70
17,great,68
2,hotel,58
140,excellent,53


In [0]:
reviews['Rating'] = reviews['Rating'].apply(lambda x:int(x))

In [0]:
reviews['nature'] = ''
for i in range(reviews.shape[0]):
    if reviews['Rating'].iloc[i] >= 4:
        reviews['nature'].iloc[i] = 'Positive'
    elif reviews['Rating'].iloc[i] <= 2:
        reviews['nature'].iloc[i] = 'Negative'
    elif reviews['Rating'].iloc[i] == 3:
        reviews['nature'].iloc[i] = 'Neutral'
    else:
        pass

In [0]:
print('Number of Positive Reviews: ', reviews[reviews['nature'] == 'Positive'].shape[0])
print('Number of Negative Reviews: ', reviews[reviews['nature'] == 'Negative'].shape[0])
print('Number of Neutral Reviews: ', reviews[reviews['nature'] == 'Neutral'].shape[0])

Number of Positive Reviews:  238
Number of Negative Reviews:  22
Number of Neutral Reviews:  35
